# Getting data from an API

The first site I thought of when trying to get news data is from a site I use quite often, I enjoy gaming a whole lot so I thought why not learn how to get gaming news using notebook? So here it is.

The first I had to do was install the steam package in order to use their api: https://pypi.org/project/steam/#modal-close

In [1]:
from steam.webapi import WebAPI
import pandas as pd
import json


api = WebAPI(key="3EDCD536CAB17FC02E4E84D9767E4FDA")

#### SteamWebAPI has a method 'GetNewsForApp' that allows users to pull news from a specific app/game
More on https://developer.valvesoftware.com/wiki/Steam_Web_API#GetNewsForApp_.28v0001.29

In [2]:
api.ISteamNews.GetNewsForApp.doc()

"GET GetNewsForApp (v0002)\n  \n  Parameters:\n    appid                     uint32   required\n      - AppID to retrieve news for\n    count                     uint32   optional\n      - # of posts to retrieve (default 20)\n    enddate                   uint32   optional\n      - Retrieve posts earlier than this date (unix epoch timestamp)\n    feeds                     string   optional\n      - Comma-separated list of feed names to return news for\n    maxlength                 uint32   optional\n      - Maximum length for the content to return, if this is 0 the full content is returned, if it's less then a blurb is generated to fit.\n    tags                      string   optional\n      - Comma-separated list of tags to filter by (e.g. 'patchnodes')\n"

It appears that an appid is necessary in order to specify to the api on which news you are getting.

Each and every app/game in steam has their corresponding appid, this can be viewed by visiting any page in steam related to the game (https://steamcommunity.com/app/1172470: appid is at the very end).

I'll be using Apex as the game to find news on as it is one of the more popular games in the platform, thus leading to more articles that I could scrape.

### Getting all news before March 13
The method 'GetNewsForApp' given by the api only has an 'enddate' parameter and no start date, we would have to manually select out all posts before march 11 later.

*p.s sometimes running the api would result in a timeout, running it a few more times would eventually work

In [5]:
apexid = 1172470

end="2021-03-13"
end_date=pd.to_datetime(end)
end_epoch=int(end_date.timestamp())

posts = api.ISteamNews.GetNewsForApp(appid = apexid, enddate=end_epoch, count = 20)
posts

{'appnews': {'appid': 1172470,
  'newsitems': [{'gid': '4057152305731955147',
    'title': '576p и 30 FPS: Раскрыты технические особенности Apex Legends для Switch',
    'url': 'https://steamstore-a.akamaihd.net/news/externalpost/Gamemag.ru/4057152305731955147',
    'is_external_url': True,
    'author': 'Дмитрий Чистяков',
    'contents': '<img src="https://gamemag.ru/images/cache/News/News155213/be112c4f2c-2_1390x600.jpg" alt=""><p>Уже сегодня, 9-го марта, состоится выход Switch-версии <strong>Apex Legends</strong>. В преддверии релиза журналисты японского издания Famitsu взяли интервью у<strong> Respawn Entertainment </strong>и студии <strong>Panic Button</strong>, ответственной за перенос условно-бесплатной королевской битвы на гибридную консоль Nintendo.</p>',
    'feedlabel': 'Gamemag.ru',
    'date': 1615319457,
    'feedname': 'Gamemag.ru',
    'feed_type': 0,
    'appid': 1172470},
   {'gid': '4057152305731955159',
    'title': 'Apex Legends пока останется без кросспрогрессинг

It appears that news from different publishers are also included such as one from Gamemag.ru whom disseminates game news in russian. We would now use the 'feeds' parameter to limit the news sources from the steam communuty only.

In [6]:
posts = api.ISteamNews.GetNewsForApp(appid = apexid, enddate=end_epoch, count = 20, feeds ='steam_community_announcements')
posts

{'appnews': {'appid': 1172470,
  'newsitems': [{'gid': '4057152305731687442',
    'title': 'Chaos Theory Now LIVE!',
    'url': 'https://steamstore-a.akamaihd.net/news/externalpost/steam_community_announcements/4057152305731687442',
    'is_external_url': True,
    'author': 'Respawn Entertainment',
    'contents': '[img]{STEAM_CLAN_IMAGE}/38098458/b869ccc1d40913200d0939af90313f8e34caff42.jpg[/img]\n\nMad Maggie has sabotaged the Ring and ring flares are now showing up across the Apex Games in the new Ring Fury Takeover. Brave the gas for loot in Caustic’s Town Takeover. Earn free rewards during the event and shop for 24 limited-time event items. Collect all 24 event items to receive Bangalore’s Cold Steel Heirloom. Ready to embrace the chaos?',
    'feedlabel': 'Community Announcements',
    'date': 1615316533,
    'feedname': 'steam_community_announcements',
    'feed_type': 1,
    'appid': 1172470},
   {'gid': '4057152305716997307',
    'title': 'Winter Circuit #4',
    'url': 'http

In [7]:
print(type(posts))

<class 'dict'>


#### The method returned a dictionary of values
Let's unpack the posts to see how the values are stored

In [8]:
for key, value in posts.items():
    print("key: " + key)
    print(value)

key: appnews
{'appid': 1172470, 'newsitems': [{'gid': '4057152305731687442', 'title': 'Chaos Theory Now LIVE!', 'url': 'https://steamstore-a.akamaihd.net/news/externalpost/steam_community_announcements/4057152305731687442', 'is_external_url': True, 'author': 'Respawn Entertainment', 'contents': '[img]{STEAM_CLAN_IMAGE}/38098458/b869ccc1d40913200d0939af90313f8e34caff42.jpg[/img]\n\nMad Maggie has sabotaged the Ring and ring flares are now showing up across the Apex Games in the new Ring Fury Takeover. Brave the gas for loot in Caustic’s Town Takeover. Earn free rewards during the event and shop for 24 limited-time event items. Collect all 24 event items to receive Bangalore’s Cold Steel Heirloom. Ready to embrace the chaos?', 'feedlabel': 'Community Announcements', 'date': 1615316533, 'feedname': 'steam_community_announcements', 'feed_type': 1, 'appid': 1172470}, {'gid': '4057152305716997307', 'title': 'Winter Circuit #4', 'url': 'https://steamstore-a.akamaihd.net/news/externalpost/stea

It appears that the key is simply the title of what you are getting and the details the news are stored within the values.

In [9]:
for key in posts['appnews']:
    print("key: " + key)

key: appid
key: newsitems
key: count


'appid' pertains to the app or game that you have chosen to get news on, 'newsitems' contains all the information about the article, I have no idea what 'count' is.

In [10]:
newsitems = posts['appnews']['newsitems']
newsitems

[{'gid': '4057152305731687442',
  'title': 'Chaos Theory Now LIVE!',
  'url': 'https://steamstore-a.akamaihd.net/news/externalpost/steam_community_announcements/4057152305731687442',
  'is_external_url': True,
  'author': 'Respawn Entertainment',
  'contents': '[img]{STEAM_CLAN_IMAGE}/38098458/b869ccc1d40913200d0939af90313f8e34caff42.jpg[/img]\n\nMad Maggie has sabotaged the Ring and ring flares are now showing up across the Apex Games in the new Ring Fury Takeover. Brave the gas for loot in Caustic’s Town Takeover. Earn free rewards during the event and shop for 24 limited-time event items. Collect all 24 event items to receive Bangalore’s Cold Steel Heirloom. Ready to embrace the chaos?',
  'feedlabel': 'Community Announcements',
  'date': 1615316533,
  'feedname': 'steam_community_announcements',
  'feed_type': 1,
  'appid': 1172470},
 {'gid': '4057152305716997307',
  'title': 'Winter Circuit #4',
  'url': 'https://steamstore-a.akamaihd.net/news/externalpost/steam_community_announce

In [11]:
print(type(newsitems))

<class 'list'>


It appears that details of each news are stored together in a list

In [12]:
print(type(newsitems[0]))

<class 'dict'>


And the details are stored as a dictionary object

Let us now store all the necessary information into a json, I converted the date to a readable format to see the exact dates of the news articles

In [13]:
ApexNews_json = []

for i in newsitems:
    for key, value in i.items():            
        if key == "date":
            date = pd.to_datetime(value, unit='s')             
        if key == 'title':
            title = value
        if key == 'contents':
            contents = value
        if key == 'author':
            author = value
    ApexNews_json.append({
            "date": date,
            "title": title,
            "contents": contents,
            "author": author
    })
              

In [14]:
ApexNews_json

[{'date': Timestamp('2021-03-09 19:02:13'),
  'title': 'Chaos Theory Now LIVE!',
  'contents': '[img]{STEAM_CLAN_IMAGE}/38098458/b869ccc1d40913200d0939af90313f8e34caff42.jpg[/img]\n\nMad Maggie has sabotaged the Ring and ring flares are now showing up across the Apex Games in the new Ring Fury Takeover. Brave the gas for loot in Caustic’s Town Takeover. Earn free rewards during the event and shop for 24 limited-time event items. Collect all 24 event items to receive Bangalore’s Cold Steel Heirloom. Ready to embrace the chaos?',
  'author': 'Respawn Entertainment'},
 {'date': Timestamp('2021-03-05 18:07:30'),
  'title': 'Winter Circuit #4',
  'contents': '[img]{STEAM_CLAN_IMAGE}/38098458/dc1ead6f9c751727cf4f3e6dd1b5caa1a8636a64.jpg[/img]\n\nTune in on March 7th as the ALGS Winter Circuit continues with the Online Tournament #4. The competition is only getting more intense as the teams pursue their chance for a spot in the March Playoffs and a $750,000 prize pool. Learn more at: https://

In [18]:
df = pd.DataFrame(ApexNews_json)
print (df)

                  date                                              title  \
0  2021-03-09 19:02:13                             Chaos Theory Now LIVE!   
1  2021-03-05 18:07:30                                  Winter Circuit #4   
2  2021-03-04 19:30:36                      CHAOS THEORY COLLECTION EVENT   
3  2021-02-24 20:02:55                                 Server Maintenance   
4  2021-02-23 21:13:48              Anniversary Collection Event Extended   
5  2021-02-20 03:05:55                                       Last Chance!   
6  2021-02-18 13:22:16                  Exclusive “Off Ramp” Weapon Charm   
7  2021-02-16 19:37:35     COVER YOUR BRUDDAHS WITH THE GIBRALTAR EDITION   
8  2021-02-09 19:17:51  Apex Legends Anniversary Collection Event is N...   
9  2021-02-04 22:46:33          Looking Back on Two Years of Apex Legends   
10 2021-02-04 17:41:51  Apex Legends Anniversary Collection Event Trailer   
11 2021-02-02 18:48:32  Season 8: Mayhem is now live across all platfo...   

It appears that JSON can not serialize timestamps so I redid the saving part but without converting the dates to timestamps.

In [27]:
ApexNews_json = []

for i in newsitems:
    for key, value in i.items():            
        if key == "date":
            date = value           
        if key == 'title':
            title = value
        if key == 'contents':
            contents = value
        if key == 'author':
            author = value
    ApexNews_json.append({
            "date": date,
            "title": title,
            "contents": contents,
            "author": author
    })
 
df = pd.DataFrame(ApexNews_json)

with open('ApexNews_json.json', 'w') as outfile:
    json.dump(ApexNews_json, outfile)
    
print (df)

          date                                              title  \
0   1615316533                             Chaos Theory Now LIVE!   
1   1614967650                                  Winter Circuit #4   
2   1614886236                      CHAOS THEORY COLLECTION EVENT   
3   1614196975                                 Server Maintenance   
4   1614114828              Anniversary Collection Event Extended   
5   1613790355                                       Last Chance!   
6   1613654536                  Exclusive “Off Ramp” Weapon Charm   
7   1613504255     COVER YOUR BRUDDAHS WITH THE GIBRALTAR EDITION   
8   1612898271  Apex Legends Anniversary Collection Event is N...   
9   1612478793          Looking Back on Two Years of Apex Legends   
10  1612460511  Apex Legends Anniversary Collection Event Trailer   
11  1612291712  Season 8: Mayhem is now live across all platfo...   
12  1612205754                       SEASON 8: MAYHEM PATCH NOTES   
13  1612204496  Apex Legends Seaso

### Disclaimer
I noticed far too late in making this notebook that there were no news between March 11-12, in fact the latest news was on March 10 and the one before it March 6. I should have realized in the beginning that game news aren't posted that happen often, much less multiple articles daily. I was simply too enamored by the fact that I was able to use the api of a platform that I've been for using for years.

I hope you would forgive me sir for not being able to comply to the specified dates for the news as I tried to limit the number of news to 20 instead. My sincerest apologies.